# Install FRED API Package

In [2]:
!pip install fredapi

# Enter your API key

In [3]:
from fredapi import Fred
fred = Fred(api_key='5be539e6d010f6d4b2d3814a6581ee8f')

# Load Required Packages

In [6]:
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype
import plotly.express as px
import plotly.graph_objects as go

from prophet import Prophet
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric
from prophet.diagnostics import cross_validation

from sklearn.metrics import mean_squared_error, mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

# Download the time series variables from FRED

In [5]:
series_mapping = {
    "USEPUINDXD": "Economic Policy Uncertainty Index for United States",
    "WLEMUINDXD": "Equity Market-related Economic Uncertainty Index"
    }

start_date = "1985-01-01"
end_date = pd.to_datetime("today").strftime("%Y-%m-%d")

data = {}
for series_id, series_name in series_mapping.items():
    data[series_name] = fred.get_series(series_id, start_date=start_date, end_date=end_date)

df = pd.DataFrame(data)
df = df.dropna()
df = df.reset_index()
df = df.rename(columns={'index': 'Date'})
df.head()

,Date,Economic Policy Uncertainty Index for United States,Equity Market-related Economic Uncertainty Index
0,1985-01-01,103.83,272.59
1,1985-01-02,296.43,972.76
2,1985-01-03,56.06,735.81
3,1985-01-04,118.45,466.45
4,1985-01-05,88.56,697.47


# Initial Visualization of Time Series Data

In [9]:
fig = px.line(df, x='Date', y='Equity Market-related Economic Uncertainty Index', title='Equity Market-related Economic Uncertainty Index Over Time')
fig.show()

# Set y variable

In [10]:
df = df.rename(columns={'Equity Market-related Economic Uncertainty Index': 'y'})
df.head()

,Date,Economic Policy Uncertainty Index for United States,y
0,1985-01-01,103.83,272.59
1,1985-01-02,296.43,972.76
2,1985-01-03,56.06,735.81
3,1985-01-04,118.45,466.45
4,1985-01-05,88.56,697.47


# Create Time Series Features

In [17]:
cat_type = CategoricalDtype(categories=['Monday','Tuesday',
                                        'Wednesday',
                                        'Thursday','Friday',
                                        'Saturday','Sunday'],
                            ordered=True)

def create_features(df, label=None):
    df = df.copy()
    df['date'] = df['Date']
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['weekday'] = df['date'].dt.day_name()
    df['weekday'] = df['weekday'].astype(cat_type)
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    df['date_offset'] = (df.date.dt.month*100 + df.date.dt.day - 320)%1300

    df['season'] = pd.cut(df['date_offset'], [0, 300, 602, 900, 1300],
                          labels=['Spring', 'Summer', 'Fall', 'Winter']
                   )
    X = df[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear','weekday',
           'season']]
    if label:
        y = df[label]
        return X, y
    return X

In [18]:
X, y = create_features(df, label='y')
features_and_target = pd.concat([X, y], axis=1)
features_and_target.head()

,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,weekday,season,y
0,0,1,1,1,1985,1,1,1,Tuesday,Winter,272.59
1,0,2,1,1,1985,2,2,1,Wednesday,Winter,972.76
2,0,3,1,1,1985,3,3,1,Thursday,Winter,735.81
3,0,4,1,1,1985,4,4,1,Friday,Winter,466.45
4,0,5,1,1,1985,5,5,1,Saturday,Winter,697.47


# Visualize Daily and Seasonal Distributions

In [19]:
fig = px.box(features_and_target, x='weekday', y='y', color='season',
             labels={'Target': 'Economic Uncertainty Index'},
             title='Economic Uncertainty by Day of Week')
fig.update_layout(xaxis_title='Day of Week', yaxis_title='Economic Uncertainty Index')
fig.show()

# Fit Model with Target Variable

In [20]:
# Create datetime column
features_and_target.rename(columns={'dayofmonth': 'day'}, inplace=True)
features_and_target['ds'] = pd.to_datetime(features_and_target[['year', 'month', 'day']])

df = features_and_target[['ds', 'y']]

# Initialize and fit the model
model = Prophet()
model.fit(df)

# Create a future dataframe for predictions (365 days)
future = model.make_future_dataframe(periods=365)

# Generate Forecast
forecast = model.predict(future)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpt4rafwxy/8picfgwl.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpt4rafwxy/i93upbis.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=36246', 'data', 'file=/tmp/tmpt4rafwxy/8picfgwl.json', 'init=/tmp/tmpt4rafwxy/i93upbis.json', 'output', 'file=/tmp/tmpt4rafwxy/prophet_modelmbhgfoqa/prophet_model-20240301002502.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
00:25:02 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
00:25:10 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


# Visualize the Prophet Forecast

In [21]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['ds'], y=df['y'], mode='markers', name='Actual',marker_size=2))
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Forecast'))
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], fill='tonexty', mode='none', name='Upper Confidence Interval'))
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], fill='tonexty', mode='none', name='Lower Confidence Interval'))

fig.update_layout(title='Equity Market-related Economic Uncertainty Forecast with Prophet', xaxis_title='Date', yaxis_title='Economic Uncertainty Index')

fig.show()